In [1]:
import requests
import pandas as pd 
import numpy as np
from bs4 import BeautifulSoup 

In [2]:
#collection url extension

extension = 'nfh'

art_number = 50

art_number_div = int(art_number/50)

## get token ids 

token_id_list = []

address_list = []

for i in range(0, art_number_div):

    params = {
        'collection' : extension,
        'offset' : i*50,
        'limit' : '50',
        'order_direction':'desc'
            }

    url = 'https://api.opensea.io/api/v1/assets'

    r = requests.request("GET", url ,params = params)
    
    for j in range(0,50):
        token_id_list.append(r.json()['assets'][j].get('token_id'))
        address_list.append(r.json()['assets'][j].get('asset_contract').get('address'))
        
trait_type_list = []
value_list = []
trait_type_list2 = []
value_list2 = []


num_sales_list = []
symbol_list = []
ending_price_list = []

columns = []
        
for i in range(0,len(token_id_list)):
    
    params_assets = {
        'order_direction' : 'desc',
        'asset_contract_address':address_list[i],
        'token_ids':token_id_list[i],
        'offset':'0',
        'limit':'50'
    }
    
    url_assets = 'https://api.opensea.io/api/v1/assets'
    
    r_assets = requests.get(url_assets ,params = params_assets)

    
    trait_count = len(r_assets.json()['assets'][0].get('traits'))
    
    for j in range(0,trait_count):
        
        t_type = r_assets.json()['assets'][0].get('traits')[j].get('trait_type')
        
        try:
            trait_type_list.append(t_type)
        
        except Exception:
            trait_type_list.append('N/A')
        
        if t_type not in columns:
            columns.append(t_type)
        
        value_list.append(r_assets.json()['assets'][0].get('traits')[j].get('value'))

    
    
    url_events = 'https://api.opensea.io/api/v1/events'

    params_events = {"token_id":token_id_list[i],
          'asset_contract_address':address_list[i]}

    r_events = requests.get(url_events,params=params_events)
    
    try:
        num_sales_list.append(int(r_events.json().get('asset_events')[0].get('asset').get('num_sales')))
    
    except Exception:
        num_sales_list.append('N/A')
    
    try:
        symbol_list.append(r_events.json().get('asset_events')[0].get('payment_token').get('symbol'))
    except Exception:
        symbol_list.append('N/A')
    
    try:
        ending_price_list.append(int(r_events.json().get('asset_events')[0].get('top_bid').replace('0',''))/10000)
    except Exception:
        ending_price_list.append('N/A')
        

        
df = np.zeros(shape =(art_number, 4+len(columns)))

column_list = ["TokenId","NumSales","Symbol","Price"]
columns
for i in columns:
    column_list.append(i)
column_list

df = pd.DataFrame(df, columns=column_list)
df['TokenId'] = token_id_list
df['NumSales'] = num_sales_list
df['Symbol'] = symbol_list
df['Price'] = ending_price_list

count = 0

value = 0

for i in range(0,art_number):
    
    for j in range(0,len(columns)):
        
        try:
            num = columns.index(trait_type_list[value])
            df[columns[num]].loc[count] = value_list[value]
        
        except Exception:
            value -=1
            pass
   
            
                
        value += 1
    
    count += 1

C:\Users\baris.baturay\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [3]:
df

,TokenId,NumSales,Symbol,Price,Identity,Faction,Headgear,Background,Mouth,Base,Eyes,Suit
0,8887,0,ETH,N/A,Male,Hero,Hawk,Green,Hey,Trippy,Angry,Boomerang
1,8886,1,ETH,N/A,Male,Hero,Tripshot,Purple,Smiley,Beige,Robin,Aureate armour
2,8885,1,ETH,N/A,Male,Hero,Swoop,Pink,Ciggy,Purple,Scope,Midnight
3,8884,1,ETH,N/A,Male,Hero,Hawk,Turquoise,Hey,Beige,Flare,Midnight
4,8883,0,WETH,N/A,Female,Hero,Web,Yellow,Hey,Leopard,Scope,Vixen
5,8882,0,ETH,N/A,Male,Villain,Montana,Green,Trope,Black,Bumble,Beluga
6,8881,1,ETH,N/A,Male,Hero,Goblin,Pink,Smiley,Black,Architect,Desert trawler
7,8880,0,N/A,N/A,Male,Villain,Rampage,Orange,Schmoke,Red,Torch,Yard
8,8879,0,WETH,N/A,Male,Hero,KungFu,Blue,Whistle,Brown,Clops,Mr Cool
9,8878,0,WETH,N/A,Male,Hero,Bananda,Green,Sulk,Beige,Hmmm,Mystery


In [4]:
df = np.zeros(shape =(art_number, 4+len(columns)))

column_list = ["TokenId","NumSales","Symbol","Price"]
columns
for i in columns:
    column_list.append(i)
column_list

df = pd.DataFrame(df, columns=column_list)
df['TokenId'] = token_id_list
df['NumSales'] = num_sales_list
df['Symbol'] = symbol_list
df['Price'] = ending_price_list

count = 0

value = 0

for i in range(0,art_number):
    
    for j in range(0,len(columns)):
        
        try:
            num = columns.index(trait_type_list[value])
            df[columns[num]].loc[count] = value_list[value]
        
        except Exception:
            value -=1
            pass
   
            
                
        value += 1
    
    count += 1

C:\Users\baris.baturay\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [5]:
params = {
        'collection' : extension,
        'offset' : 0,
        'limit' : '50',
        'order_direction':'desc'
            }

url = 'https://api.opensea.io/api/v1/assets'

r = requests.request("GET", url ,params = params)

In [11]:
r.json().get('assets')[2].get('token_id')

'8885'

[{'id': 533114,
  'token_id': '10000',
  'num_sales': 0,
  'background_color': None,
  'image_url': 'https://www.larvalabs.com/cryptopunks/cryptopunk10000.png',
  'image_preview_url': 'https://www.larvalabs.com/cryptopunks/cryptopunk10000.png',
  'image_thumbnail_url': None,
  'image_original_url': None,
  'animation_url': None,
  'animation_original_url': None,
  'name': 'CryptoPunk #10000',
  'description': None,
  'external_link': 'https://www.larvalabs.com/cryptopunks/details/10000',
  'asset_contract': {'address': '0xb47e3cd837ddf8e4c57f05d70ab865de6e193bbb',
   'asset_contract_type': 'non-fungible',
   'created_date': '2018-01-23T04:51:38.832339',
   'name': 'CryptoPunks',
   'nft_version': 'unsupported',
   'opensea_version': None,
   'owner': None,
   'schema_name': 'CRYPTOPUNKS',
   'symbol': 'PUNK',
   'total_supply': None,
   'description': 'CryptoPunks launched as a fixed set of 10,000 items in mid-2017 and became one of the inspirations for the ERC-721 standard. They have 